# Notion Agent

In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor notion-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
NOTION_TOKEN = "token here"
DATABASE_ID = "20f4296aca078044a20fd88ec8246774"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

In [ ]:
# #5th prompt
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# ## TASK FLOW

# 1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

# 2. If the user also wants to "write about ..." in the same request:
#     - First, call `notion_page_agent(title=...)`.
#     - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
#     - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

# ## PAGE_ID EXTRACTION

# - The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
# - You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

# ## EXAMPLE:

# If the user says:
# "Create a new page named Ichigo. write about anime bleach in my notion page."

# You must:

# 1. Call `notion_page_agent(title='Ichigo')`
# 2. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
# 3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

# ---

# If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
# """


In [ ]:
page_agent_prompt = """
You are a Notion agent responsible for managing pages within a Notion workspace. You're equipped with different tools.

Your primary tasks include:
1. **Creating Pages**: Create new pages with specified titles, content, and properties. Ensure pages are formatted according to user requests.

When performing actions:
- Always ensure page titles and content are clear and concise.
- If tool is not available to perform the action, simply say “I am unable to perform the request"
"""

In [ ]:
block_agent_prompt = """
You are a Notion agent responsible for add blocks within Notion pages. Your tasks include:
    - **Creating Blocks**: Add new blocks to a page. Ensure the blocks are created with the proper content and formatting as specified by the user.

2. FORMATTING REQUIREMENTS:
  - Follow Notion's native styling
  - Maintain consistent spacing
  - Preserve hierarchical structure
  - Apply appropriate indentation

## ERROR HANDLING:
  - If a requested block type is unsupported: Respond with ‘Block type not supported’
  - If missing required parameters: Respond with ‘Missing required information: [parameter]’
  - For unauthorized actions: Respond with ‘Permission denied’
  - If tool is not available to perform the action: Respond with ‘I am unable to perform the request’

## Important:
  - When using add_notion_number_list_block, add_notion_bulleted_list_block, and add_notion_to_do_block functions, argument must be a
      list of str. For example
      add_notion_number_list_block([‘item 1°, ‘item 2‘, ‘item 3'])
      add_notion_bulleted_list_block([‘item 1', ‘item 2', ‘item 3'])
      add_notion_to_do_block([‘item 1', ‘item 2', ‘item 3'])
  - When passing a string containing a list of item, Use comma (",") as separator.
    For example, “Chapter 1, Chapter 2, "Chapter 3" is acceptable. “Chapter 1\nChapter 2\n Chapter 3" is not.
"""

In [ ]:
import os
from typing import List
import httpx
from langchain_core.tools import tool

NOTION_API_KEY = "token here"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

@tool
def create_notion_page(page_title):
  """
  Create a new page in Notion with the specified title and content.

  args:
      page_title: The title of the new page.
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/pages'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  properties= {
        "title": [
            {
                "text": {
                    "content": page_title
                }
            }
        ]
    }
  try:
    response = httpx.post(url, headers=headers, json={
        'parent': {
            'page_id': page_id
        },
        'properties': properties
    })
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}


In [ ]:
@tool
def add_notion_heading_block(
        heading_type: str = 'heading_1',
        content: str = 'New Heading',
        is_hyperlink: bool = False,
        hyperlink_url: str = '',
        color: str = 'default'
):
  """
  Add a heading block to a Notion page with the specified content.

  args:
      context_variables: The context variables containing the API key and page ID.
      heading_type: The type of heading block to add (e.g., "heading_1', 'heading_2").
      content: The content of the heading block.
      is_hyperlink: A boolean value indicating whether the heading should be a hyperlink.
      hyperlink_url: The URL of the hyperlink.
      color: The color of the heading block.

  example usage:
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=False)
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=True, hyperlink_url='https://en.wikipedia.org/wiki/Lacinato_kale')
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  text_obj = {'type': 'text', 'text': {'content': content}}
  data = {
      'children': [{
          'object': 'block',
          'type': heading_type,
          heading_type: {
              'rich_text': [text_obj],
              'color': color # Move color inside the heading block
          }
      }]
  }

  if is_hyperlink:
    text_obj['text']['link'] = {'url': hyperlink_url}

  try:
    response = httpx.patch(url, headers=headers, json=data)
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}

In [ ]:
@tool
def add_notion_paragraph_block(
    page_id: str,
    content: str,
    is_hyperlink: bool = False,
    hyperlink_url: str = '',
    color: str = 'default'
):
    """
    Add a paragraph block to a Notion page with the specified content.

    Args:
        page_id: The ID of the Notion page where the block will be added.
        content: The content of the paragraph block.
        is_hyperlink: Whether the paragraph is a hyperlink.
        hyperlink_url: The URL if it’s a hyperlink.
        color: The color of the paragraph block.

    Example usage:
        add_notion_paragraph_block(page_id='abc123', content='Hello', is_hyperlink=False)
        add_notion_paragraph_block(page_id='abc123', content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')
    """
    url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
    headers = {
        'Authorization': f'Bearer {NOTION_API_KEY}',
        'Content-Type': 'application/json',
        'Notion-Version': '2022-06-28'
    }

    text_obj = {'type': 'text', 'text': {'content': content}}
    if is_hyperlink:
        text_obj['text']['link'] = {'url': hyperlink_url}

    data = {
        'children': [{
            'object': 'block',
            'type': 'paragraph',
            'paragraph': {
                'rich_text': [text_obj],
                'color': color
            }
        }]
    }

    try:
        response = httpx.patch(url, headers=headers, json=data)
        return response.json()
    except httpx.ReadTimeout:
        return {'error': 'Request timed out'}


In [ ]:
from notion_client import Client

import re
from langchain.tools import tool


def extract_page_id_from_notion_response(response: str) -> str:
    """
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

    This function will return the page id:
      "2104296aca0781e299b5d1e16f714891"

    If no matching page id is found, it returns an explanatory message.
    """
    # This regex looks for a URL starting with https://www.notion.so/
    # followed by any characters (representing the title) and a hyphen,
    # then a 32-character hexadecimal string (the page id).
    pattern = r"https:\/\/www\.notion\.so\/[^-\s]+-([a-f0-9]{32})"
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    else:
        return "No page id found in the response."


In [ ]:
notion_page_agent = create_react_agent(
    model=model,
    tools=[create_notion_page, extract_page_id_from_notion_response],
    prompt=page_agent_prompt,
    name="notion_page_agent"
)

In [ ]:
notion_block_agent = create_react_agent(
    model=model,
    tools=[add_notion_heading_block, add_notion_paragraph_block, extract_page_id_from_notion_response],
    prompt=block_agent_prompt,
    name="notion_block_agent"
)

# Meditation Agent

In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# PDF Loading
from langchain.document_loaders import PyPDFLoader
pdf_files = ["27_Meditation_Techniques.pdf", "how-to-meditate.pdf"]
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_documents.extend(docs)
documents = all_documents
print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs.")

Loaded 39 documents from 2 PDFs.


In [ ]:
# Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

Split into 89 chunks.


In [ ]:
# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
# Vectorstore (ChromaDB)
from langchain_chroma import Chroma as ch
persist_directory = "chroma_db"
db = ch.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
# Retriever with similarity score threshold
retriever = db.as_retriever(
    search_type="similarity", #change
    search_kwargs={"k": 4}
)

In [ ]:
# RAG Prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG LCEL chain
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from langchain.tools import tool
from typing import List
import requests
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chat_models import ChatOpenAI

# RAG Prompt Template
custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG Chain
rag_chain = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Define tool
@tool
def rag_tool(question: str) -> str:
    """
    Answers a question using a retrieval-augmented generation (RAG) approach.
    Retrieves relevant documents from a vector store and uses an LLM to generate the final answer.
    """
    return rag_chain.invoke({"question": question})


In [ ]:
from langchain_core.messages import AIMessage
from typing import Annotated
from langchain.tools import tool

@tool
def store_meditation_response_to_notion(question: Annotated[str, "User query about meditation"],
                                        title: Annotated[str, "Title of the Notion page to store the response"]) -> str:
    """
    Get an answer from the meditation assistant and store it in a Notion page.
    """
    # Step 1: Get answer from meditation_assistant
    response = meditation_assistant.invoke({"input": question})

    # Extract response content safely
    if isinstance(response, AIMessage):
        answer = response.content
    elif isinstance(response, dict) and "output" in response:
        answer = response["output"]
    else:
        raise ValueError("Unexpected response format from meditation_assistant")

    # Step 2: Create Notion page
    page_response = notion_page_agent.invoke({"title": title})

    # Step 3: Extract page_id
    if isinstance(page_response, dict) and "page_id" in page_response:
        page_id = page_response["page_id"]
    elif isinstance(page_response, AIMessage):
        # Example: "page_id = 'abc123...'"
        import re
        match = re.search(r'page_id\s*=\s*[\'"]?([\w\d]{32})[\'"]?', page_response.content)
        if not match:
            raise ValueError("Unable to extract page_id from Notion page agent response")
        page_id = match.group(1)
    else:
        raise ValueError("Unexpected response format from notion_page_agent")

    # Step 4: Add content to the Notion page
    notion_block_agent.invoke({"page_id": page_id, "content": answer})

    return f"Stored meditation answer in Notion page titled '{title}'."


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

meditation_assistant = create_react_agent(
    model=model,
    tools=[rag_tool, store_meditation_response_to_notion],
    prompt="You are a helpful Meditation assistant. Your job is to use RAG tool to Answers questions using the provided PDF documents. ",
    name="meditation_assistant"
)


In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to do breathing exercises?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to do breathing exercises?', additional_kwargs={}, response_metadata={}, id='93e1ac76-78c5-4f24-9905-ddc9ddb5e8bd'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rnd2aVHmRrOYAWMlAa3i5jlP', 'function': {'arguments': '{"question":"How to do breathing exercises?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 159, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiLtQgBIvomVPcSl34Ab9Hy4JM9hq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--f117d390-ed35-47c6-835f-eeae2fc91df0-0', tool_calls=[{'name': 'rag_tool'

# Supervisor Agent

In [ ]:
#5th prompt
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

## TASK FLOW

1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

2. If the user also wants to "write about ..." in the same request:
    - First, call `notion_page_agent(title=...)`.
    - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
    - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
- You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

## EXAMPLE:

If the user says:
"Create a new page named Ichigo. write about anime bleach in my notion page."

You must:

1. Call `notion_page_agent(title='Ichigo')`
2. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

---

If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
"""


new

In [ ]:
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# You have access to the following agents:
# - `notion_page_agent`: Creates new Notion pages.
# - `notion_block_agent`: Adds content blocks to an existing Notion page.
# - `meditation_assistant`: Answers questions related to meditation using PDF-based context through a RAG system.

# ---

# ## TASK FLOW

# ### 1. Page Creation

# If the user says: "Create a new page named <title>", call:
# `notion_page_agent(title=...)`

# ---

# ### 2. Page Creation + Custom Content

# If the user also wants to "write about ..." in the same request:
# 1. Call `notion_page_agent(title=...)`
# 2. The `notion_page_agent` uses `extract_page_id_from_notion_response` to return a `page_id`.
# 3. Then call:
#    `notion_block_agent(page_id=..., content=...)`

# ---

# ### 3. Meditation Queries and Storing Answers in Notion

# If the user's request is related to meditation (e.g., breathing, mindfulness, techniques, stress relief, etc.):
# - Call:
#   `meditation_assistant(question=...)`

# If the user also wants to **record the meditation answer in a Notion page**:
# 1. First call: `meditation_assistant(question=...)`
# 2. Capture the result (e.g., `"One effective meditation is box breathing..."`)
# 3. Then call: `notion_page_agent(title=...)` to create a Notion page
# 4. Receive `page_id` from the page agent
# 5. Then call:
#    `notion_block_agent(page_id=..., content=<meditation_assistant response>)`

# ---

# ## PAGE_ID EXTRACTION

# - The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
# - You do NOT need to parse the URL manually.
# - Always use the `page_id` returned from `notion_page_agent` for `notion_block_agent`.

# ---

# ## EXAMPLES

# **Example 1**
# User says:
# "Create a new page named Ichigo. Write about anime bleach."

# You must:
# 1. Call: `notion_page_agent(title='Ichigo')`
# 2. Get: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
# 3. Call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

# ---

# **Example 2**
# User says:
# "Tell me a meditation technique and save it in a Notion page titled Calm Mind."

# You must:
# 1. Call: `meditation_assistant(question='Tell me a meditation technique')`
# 2. Assume it responds: `"One effective method is box breathing..."`
# 3. Call: `notion_page_agent(title='Calm Mind')`
# 4. Receive `page_id`
# 5. Call: `notion_block_agent(page_id=..., content='One effective method is box breathing...')`

# ---

# ## ERROR HANDLING

# - If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
# - If any tool is unavailable to complete the request, respond with:
#   “I am unable to perform the request.”
# """


In [ ]:
# #5th prompt
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# ## TASK FLOW

# 1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

# 2. If the user also wants to "write about ..." in the same request:
#     - First, call `notion_page_agent(title=...)`.
#     - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
#     - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

# 3. Meditation Queries and Storing Answers in Notion

# If the user's request is related to meditation (e.g., breathing, mindfulness, techniques, stress relief, etc.):
# - Call:
#   `meditation_assistant(question=...)`

# If the user also wants to **record the meditation answer in a Notion page**:
# 1. First call: `meditation_assistant(question=...)`
# 2. Capture the result (e.g., `"One effective meditation is box breathing..."`)
# 3. Then call: `notion_page_agent(title=...)` to create a Notion page
# 4. Receive `page_id` from the page agent
# 5. The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
# 6. Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

# ## PAGE_ID EXTRACTION

# - The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
# - You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

# ## EXAMPLE:

# If the user says:
# "Tell me a meditation technique.Create a Notion page called Calm Mind 3. Then record the meditation technique in the same notion page."

# You must:

# 1. Call meditation_assistant(question="Tell me about meditation technique")
# 2. Capture the result (e.g., `"One effective meditation is box breathing..."`)
# 3. Call `notion_page_agent(title='Calm Mind 3')`
# 4. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
# 5. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content=meditation_assistant(question="Tell me about meditation technique"))`

# ---

# If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
# """


In [ ]:
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# You have access to the following agents:
# - `notion_page_agent`: Creates a new Notion page with the given title and returns the `page_id` using `extract_page_id_from_notion_response`.
# - `notion_block_agent`: Adds content blocks to a Notion page using the `page_id`.
# - `meditation_assistant`: Answers meditation-related questions using a PDF-based context.

# ---

# ## TASK FLOW

# ### 1. Creating a Notion Page
# If the user says something like:
# **"Create a new page named <title>"**, then:
# - Call: `notion_page_agent(title=...)`

# ---

# ### 2. Writing Custom Content to a Notion Page
# If the user says:
# **"Create a new page named <title> and write about <some topic>"**, then:
# 1. Call: `notion_page_agent(title=...)`
# 2. Extract `page_id` from the result.
# 3. Then call: `notion_block_agent(page_id=..., content='<some topic>')`

# ---

# ### 3. Meditation Queries and Recording the Answer in Notion
# If the user asks a **meditation-related question**, such as:
# - "Tell me a meditation technique"
# - "How to reduce stress with breathing?"
# - "Guide me through a mindfulness practice"

# Then:
# 1. Call: `meditation_assistant(question=...)`
# 2. Store the result as `meditation_answer`

# If the user also says to save it in a Notion page (e.g., **"Save this in a Notion page called <title>"**):
# 3. Call: `notion_page_agent(title=...)`
# 4. Extract `page_id` from the result
# 5. Call: `notion_block_agent(page_id=..., content=meditation_answer)`

# ---

# ## PAGE_ID EXTRACTION

# - The `notion_page_agent` tool returns the `page_id` using the `extract_page_id_from_notion_response`.
# - Do **not** manually parse any URLs.
# - Use the returned `page_id` directly when calling `notion_block_agent`.

# ---

# ## EXAMPLE FLOW

# **User says:**
# "Tell me a meditation technique. Create a Notion page titled Calm Mind 3 and record the technique there."

# You must:
# 1. Call: `meditation_assistant(question='Tell me a meditation technique')`
# 2. Store the result (e.g., `meditation_answer = "One effective meditation is box breathing..."`)
# 3. Call: `notion_page_agent(title='Calm Mind 3')`
# 4. Get `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
# 5. Call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content=meditation_answer)`

# ---

# ## ERROR HANDLING

# - If `notion_page_agent` fails to return a `page_id`, ask the user to confirm or specify the page manually.
# - If any tool is not available, respond with: **"I am unable to perform the request."**
# """


In [ ]:
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# You have access to the following agents:
# - `notion_page_agent`: Creates a new Notion page and returns a `page_id` using `extract_page_id_from_notion_response`.
# - `notion_block_agent`: Adds content blocks to a Notion page using the `page_id`.
# - `meditation_assistant`: Answers meditation-related queries using RAG on provided documents.

# ---

# ## TASK FLOW

# ### 1. Creating a Notion Page
# If the user says something like:
# > "Create a new page named <title>"

# Then:
# - Call: `notion_page_agent(title=...)`

# ---

# ### 2. Creating a Page and Writing Custom Content
# If the user says something like:
# > "Create a new page named <title> and write about <something>"

# Then:
# 1. Call: `notion_page_agent(title=...)`
# 2. Extract the `page_id` from the result
# 3. Call: `notion_block_agent(page_id=..., content='<something>')`

# ---

# ### 3. Handling Meditation Queries
# If the user asks a question related to meditation (e.g., breathing, mindfulness, focus, relaxation):

# 1. Call: `meditation_assistant(question=...)`
# 2. Store the returned content as `meditation_answer`

# If the user also wants to **store the meditation answer in a Notion page**, such as:
# > "Tell me a breathing technique and store it in a Notion page titled Calm Mind"

# Then:
# 3. Call: `notion_page_agent(title='Calm Mind')`
# 4. Extract the `page_id` from the result
# 5. Call: `notion_block_agent(page_id=..., content=meditation_answer)`

# ---

# ## PAGE_ID EXTRACTION

# - The `notion_page_agent` tool uses `extract_page_id_from_notion_response` to return the `page_id`.
# - Do not parse URLs manually — use the `page_id` directly from the response.

# ---

# ## EXAMPLE

# If the user says:
# > "Give me a meditation technique and save it in a Notion page called 'Focus Practice'"

# Then:
# 1. Call: `meditation_assistant(question='Give me a meditation technique')`
# 2. Store result as: `meditation_answer = "One effective technique is box breathing..."`

# 3. Call: `notion_page_agent(title='Focus Practice')`
# 4. Receive: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
# 5. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content=meditation_answer)`

# ---

# ## ERROR HANDLING

# - If `notion_page_agent` fails to return a `page_id`, ask the user to confirm or specify the page manually.
# - If any tool is unavailable or fails, respond with: **"I am unable to perform the request."**
# """


In [ ]:
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

## TASK FLOW

### 1. Create a Notion Page
If the user says:
> "Create a new page named <title>"

Call:
- `notion_page_agent(title=...)`

---

### 2. Create a Page and Write Arbitrary Content
If the user says:
> "Create a new page named <title> and write about <something>"

Then:
1. Call: `notion_page_agent(title=...)`
2. Extract `page_id` from the result
3. Call: `notion_block_agent(page_id=..., content='<something>')`

---

### 3. Handle Meditation-Related Requests
If the user’s request is related to meditation (e.g., they say: “Tell me about meditation techniques” or “Explain breathing exercises”), and they also mention a page to write it in, follow this flow:

1. Call: `meditation_assistant(question=...)`
2. Capture the assistant's response (e.g., `"Box breathing is a technique where you inhale for 4 seconds..."`)
3. Call: `notion_page_agent(title=...)` with the page title the user provides (e.g., `Meditate`)
4. Extract `page_id` from the result
5. Call: `notion_block_agent(page_id=..., content=meditation_answer)`

---

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool returns the `page_id` using `extract_page_id_from_notion_response`.
- You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

---

## EXAMPLE

If the user says:
> "Tell me about meditation techniques. Create a page called Meditate and write in it."

Then you must:
1. Call: `meditation_assistant(question="Tell me about meditation techniques")`
2. Store the result as `meditation_answer`
3. Call: `notion_page_agent(title='Meditate')`
4. Extract: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
5. Call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content=meditation_answer)`

---

## ERROR HANDLING

- If `notion_page_agent` fails to return a `page_id`, ask the user to confirm or specify the page manually.
- If any tool is unavailable or the action is unsupported, respond: “I am unable to perform the request.”
"""


In [ ]:
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API and route user tasks to the appropriate agents.

## TASK FLOW

### 1. Create a new Notion page
- If the user says "Create a new page named <title>", call:
    `notion_page_agent(title=...)`

### 2. Write content to a Notion page
- If the user also wants to "write about ..." in the same request:
    1. Call: `notion_page_agent(title=...)`
    2. Receive `page_id` from the response (no need to extract manually).
    3. Then call: `notion_block_agent(page_id=..., content=...)`

### 3. Meditation Queries
- If the user’s request is about meditation (e.g., techniques, mindfulness, stress relief), and they **do NOT** mention Notion:
    - Call: `meditation_assistant(question=...)`

### 4. Meditation Queries + Notion Integration
If the user wants a meditation-related question answered and also stored in Notion:
- Example: "Tell me about meditation techniques. Create a page called 'Peaceful Mind' and write in it."

Perform the following:
1. Call: `meditation_assistant(question=...)`
2. Capture the response (this is the content to be written).
3. Call: `notion_page_agent(title='Peaceful Mind')`
4. Receive the response: `page_id = ...`
5. Then call: `notion_block_agent(page_id=..., content=...)` using the meditation answer as `content`.

---

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool will return the `page_id` using `extract_page_id_from_notion_response`.
- You do **not** need to parse the URL or extract it manually.

---

## EXAMPLES

### Example 1: Normal Notion Write
Input: "Create a new page named Ichigo. write about anime bleach in my notion page."
Steps:
1. Call `notion_page_agent(title='Ichigo')`
2. page_id = "2104296aca0781f08ef8caf4bf9ad809"
3. Call `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

### Example 2: Meditation + Notion Write
Input: "Tell me about meditation techniques. Create a page called 'Abra' and write in it."
Steps:
1. Call `meditation_assistant(question='Tell me about meditation techniques')`
2. Capture the answer (e.g., "Box breathing involves 4-second inhale...")
3. Call `notion_page_agent(title='Abra')`
4. page_id = "2104296aca0781f08ef8caf4bf9ad809"
5. Call `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='Box breathing involves...')`

---

## ERROR HANDLING

- If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
"""


In [ ]:
#5th prompt
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

## TASK FLOW

1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

2. If the user also wants to "write about ..." in the same request:
    - First, call `notion_page_agent(title=...)`.
    - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
    - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

3. If the user asks a question related to meditation (e.g., meditation techniques, mindfulness, breathing, relaxation, etc.):
    - First, call `meditation_assistant(question=...)` to retrieve the meditation-related answer.
    - If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.
    - If the user also wants to "write about ..." in the same request:
        - First, call `notion_page_agent(title=...)`.
        - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
        - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=meditation_assistant(question=...))`.

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
- You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

## EXAMPLE:

If the user says:
"Tell me about meditation techniques. Create a page called Kadabra and write in it."

You must:

1. Call `meditation_assistant(question='Tell me about meditation techniques')`
2. Capture the result (e.g., `"Box breathing involves 4-second inhale..."`)
3. Call `notion_page_agent(title='Ichigo')`
2. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content=meditation_assistant(question='Tell me about meditation techniques'))`

---

If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
"""


In [ ]:
delegate_agent_prompt = """
You are a Notion Supervisor Agent that orchestrates between three specialized agents:
1. Notion Page Agent - Creates pages
2. Notion Block Agent - Adds content to pages
3. Meditation Assistant - Provides meditation knowledge

## TASK DECISION TREE

1. PAGE CREATION WITH MEDITATION CONTENT:
   - If user requests to create a page AND include meditation content:
     a) First call `notion_page_agent(title="...")` to create the page
     b) Extract the `page_id` from the response
     c) Call `meditation_assistant(query="...")` for meditation content
     d) Pass both `page_id` and meditation content to `notion_block_agent`

2. STANDARD PAGE CREATION:
   - If only page creation is requested:
     a) Call `notion_page_agent(title="...")`
     b) Optional: Ask if user wants to add content

3. MEDITATION QUERIES:
   - If only meditation information is requested:
     a) Call `meditation_assistant(query="...")`
     b) Optional: Ask if user wants to save this to Notion

4. CONTENT ADDITION TO EXISTING PAGE:
   - If content needs to be added to an existing page:
     a) Verify `page_id` is available
     b) For meditation content: First call `meditation_assistant`
     c) Call `notion_block_agent(page_id="...", content="...")`

## WORKFLOW RULES

1. PAGE ID HANDLING:
   - Always extract `page_id` from `notion_page_agent` responses
   - Never ask users for page IDs manually

2. CONTENT FORMATTING:
   - Ensure meditation content is properly formatted before sending to block agent
   - Break long content into multiple blocks if needed

3. ERROR RECOVERY:
   - If page creation fails: Notify user and suggest alternatives
   - If meditation query fails: Ask for clarification
   - If block addition fails: Retry with simplified content

## EXAMPLES

Example 1: Combined Request
User: "Create a meditation page called Mindfulness and write about breathing techniques"
Steps:
1. notion_page_agent(title="Mindfulness") → gets page_id
2. meditation_assistant(query="breathing techniques") → gets content
3. notion_block_agent(page_id=..., content=...)

Example 2: Information Only
User: "Tell me about Zen meditation"
Steps:
1. meditation_assistant(query="Zen meditation")
2. (Optional) Ask if user wants to save this information

Example 3: Page Only
User: "Create a page called My Practice"
Steps:
1. notion_page_agent(title="My Practice")
2. Ask if user wants to add content later

## OUTPUT REQUIREMENTS

- Always confirm successful operations to the user
- For meditation content, ensure it's:
  - Properly structured with headings
  - Broken into digestible paragraphs
  - Includes practical exercises when relevant
"""

In [ ]:
supervisor = create_supervisor(
    agents=[notion_page_agent, notion_block_agent, meditation_assistant],

    model=model,
    prompt=delegate_agent_prompt
).compile()



In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a meditation page called Mindfulness101 and write about breathing techniques."

            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a meditation page called Mindfulness101 and write about breathing techniques.', additional_kwargs={}, response_metadata={}, id='537ca484-3ab0-466c-9525-e570bd54c7df'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3hmRJAP4PJ988iaULTif8CnA', 'function': {'arguments': '{"title":"Mindfulness101"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 726, 'total_tokens': 746, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiLw03QqvdzEMzgpAqNiYD2FewPV6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a meditation page called Mindfulness11 and write about breathing techniques."

            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a meditation page called Mindfulness11 and write about breathing techniques.', additional_kwargs={}, response_metadata={}, id='ec8baccd-5fbb-4fa6-bc74-840bfcc1f40f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9MoXD0oKPyLZJgRJzvS44DbJ', 'function': {'arguments': '{"title":"Mindfulness11"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 726, 'total_tokens': 746, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiFoGAbMABfH0gHaQXQJq0sYm594f', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--f

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Unohana. write a paragraph about anime character Unohana from bleach in the same notion page."

            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Unohana. write a paragraph about anime character Unohana from bleach in the same notion page.', additional_kwargs={}, response_metadata={}, id='43ef1c66-98ec-4856-8438-3b334733a028'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_89atYRuE61DFETXF6YH5FkNL', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 466, 'total_tokens': 480, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiF8jjZgv6HVK8ZD13Wp94uULdmmL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor',

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Tell me about meditation techniques. Create a page called Lumine1 and write in it."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Tell me about meditation techniques. Create a page called Lumine1 and write in it.', additional_kwargs={}, response_metadata={}, id='e9c1ba83-80ff-4ff1-be01-af90448353e4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DcAmIqNBZWwxVvIbpqzMfjNf', 'function': {'arguments': '{"question":"Tell me about meditation techniques."}', 'name': 'transfer_to_meditation_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 679, 'total_tokens': 703, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiEqasWOohT5GEY63PldXKM605VA0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, n

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a page called Kadabra1 and write about pokemon kadabra in it."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a page called Kadabra1 and write about pokemon kadabra in it.', additional_kwargs={}, response_metadata={}, id='ff5d5e03-82f4-4887-93bc-4da072d12da6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_89A2BVvAp9RD20evhmVhe9i5', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 775, 'total_tokens': 789, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiEhyLsEHbr6BgUbmrVS2UlfbY93P', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--39e5d032-9bbb-4eca-935a-af3e3c013052-0'

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Tell me about meditation techniques. Create a page called Kadabra and write in it."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Tell me about meditation techniques. Create a page called Kadabra and write in it.', additional_kwargs={}, response_metadata={}, id='a3919a08-71ec-4531-9d48-26043051e1f9'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dyVGDPHMWYo0Uk5lRWKGDMzw', 'function': {'arguments': '{}', 'name': 'transfer_to_meditation_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 776, 'total_tokens': 791, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiEgZUbyKK34T4S4dhBQrQngrK7jk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--01d7215e-9816-4eca-983

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Tell me about meditation techniques. Create a page called Abra and write in it."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Tell me about meditation techniques. Create a page called Abra and write in it.', additional_kwargs={}, response_metadata={}, id='6ef4a886-ac68-45a1-ad63-9777cb97fd2c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QGevr8V1QL62pfXOP8bvUwV7', 'function': {'arguments': '{"question":"Tell me about meditation techniques."}', 'name': 'transfer_to_meditation_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 672, 'total_tokens': 696, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiEYAwGjnv3YHGB8CbgS4dPvU3ccZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a notion page called Medicham 1 .Write about meditation technique in the same page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a notion page called Medicham 1 .Write about meditation technique in the same page.', additional_kwargs={}, response_metadata={}, id='fe37fe90-11bf-420a-a80f-7df1f49d6ce6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4buyOt5ci4qzSonyibhf2JRt', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 749, 'total_tokens': 763, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiEI7jTD7QdUGwOvtDG9NSf6qxnMl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--448a599f-7a78-48c

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Tell me a meditation technique.Create a Notion page called Mind 1. Then record the meditation technique in the same notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Tell me a meditation technique.Create a Notion page called Mind 1. Then record the meditation technique in the same notion page.', additional_kwargs={}, response_metadata={}, id='fcf58df1-0a25-4625-85e9-278d9a0cf464'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lZqauIJkUyFz9SDVHesW3ioc', 'function': {'arguments': '{"question":"Tell me about meditation technique"}', 'name': 'transfer_to_meditation_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 719, 'total_tokens': 742, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiE96dJYSuFLVbBxZ2NAsNtkIBRHH', 'service_tier': 'default', 'finish

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Chad Sadooo1. write 3 sentences about anime character Chad from bleach in the same notion page."
                #"Tell me a meditation technique.Create a Notion page called Calm Mind 1. Then record the meditation technique in the same notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Chad Sadooo1. write 3 sentences about anime character Chad from bleach in the same notion page.', additional_kwargs={}, response_metadata={}, id='e9161273-8979-475d-81b0-88dd595f8105'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4wCoFlb2FnOZXwK9SeTAUPjd', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 469, 'total_tokens': 483, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiDYENSlRe0U3HPWFEoatX6GFzWNO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Chad Sado. write 3 sentences about anime character Chad from bleach in the same notion page."
                #"Tell me a meditation technique.Create a Notion page called Calm Mind 1. Then record the meditation technique in the same notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Chad Sado. write 3 sentences about anime character Chad from bleach in the same notion page.', additional_kwargs={}, response_metadata={}, id='05fc0123-a851-483e-a410-88d454736efc'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IXtwFOwcWnskMlIEjvC7DDPZ', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 772, 'total_tokens': 786, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiDQyPnzI9Y8o1htsqy7j01zBU49R', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', 

In [ ]:
delegate_agent_prompt = """
You are a Notion Supervisor Agent that strictly enforces these workflows:

1. FOR MEDITATION PAGE CREATION:
   - FIRST call `notion_page_agent(title="...")` and WAIT for page_id
   - THEN call `meditation_assistant(query="...")` and WAIT for content
   - FINALLY call `notion_block_agent(page_id=page_id, content=content)`

2. CONTENT VALIDATION:
   - Verify meditation content is non-empty before sending to block agent
   - If content is empty, retry meditation_assistant with refined query

3. ERROR HANDLING:
   - If block addition fails, RETRY with simplified content
   - Never proceed to next step without successful completion
   - Report EXACTLY where failures occur

4. OUTPUT FORMAT:
   - Confirm EACH step completion
   - Show preview of content being added
   - Provide final page URL
"""

In [ ]:
# Add this to your supervisor stream call
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a page called Lumine1 and write about meditation techniques"
            )
        ],
    },
    {
        "recursion_limit": 150,
        "debug": True  # Enable verbose logging
    },
):
    print(chunk)
    print("---")  # Add separator for clarity

{'supervisor': {'messages': [HumanMessage(content='Create a page called Lumine1 and write about meditation techniques', additional_kwargs={}, response_metadata={}, id='b4b49f7e-5c6d-46aa-8e5c-3a1e9dddbc2c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HQirCLcP6FnLwvPbKSjBHAlB', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 724, 'total_tokens': 738, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiFyUKM1gibk1fq3Aha78cujggJkl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--3e908045-2e1b-46b2-b239-b640f9c2b994-0', 

In [ ]:
extract_page_id_from_notion_response("https://www.notion.so/Lumine1-2124296aca0781b78197c2f6dea347e9?source=copy_link")

'2124296aca0781b78197c2f6dea347e9'

In [ ]:
# Test 1: Page creation only
# Test Notion Page Agent
page_response = notion_page_agent.invoke({
    "messages": [HumanMessage(content="Create a page called TestPage")]
})

print(page_response)  # Check for page_id

# Test 2: Meditation content only
# meditation_content = meditation_assistant(query="meditation techniques")
meditation_content = meditation_assistant.invoke({
    "messages": [HumanMessage(content="meditation techniques")]
})
print(meditation_content)  # Should be plain text

# Test 3: Block addition
block_response = notion_block_agent.invoke({
    "messages": [HumanMessage(page_id=extract_page_id_from_notion_response("https://www.notion.so/Lumine1-2124296aca0781b78197c2f6dea347e9?source=copy_link"),
    content="Test content")]
})
# block_response = notion_block_agent(
#     page_id="2124296aca0781b78197c2f6dea347e9",  # Use a real ID
#     content="Test content"
# )
print(block_response)  # Check for success

{'messages': [HumanMessage(content='Create a page called TestPage', additional_kwargs={}, response_metadata={}, id='9fae003a-27ce-41c8-bdec-82ab8b2e1281'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UqKM1PnTwOjiHELljcrBFpkX', 'function': {'arguments': '{"page_title":"TestPage"}', 'name': 'create_notion_page'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 367, 'total_tokens': 385, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiGGRWE0CQBgqKjCBGSFilENhA5C5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='notion_page_agent', id='run--84e21ed7-d1eb-4c34-ac7d-2d088a9f9f46-0', tool_calls=[{'name': 'create_noti

In [ ]:
meditation_content = meditation_assistant.invoke({
    "messages": [HumanMessage(content="explain breathing exerciese")]
})
print(meditation_content["messages"])

[HumanMessage(content='explain breathing exerciese', additional_kwargs={}, response_metadata={}, id='14a1841d-6691-4053-9433-c4cf591f6892'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kAD57o56GloaNw176J5zqZ0S', 'function': {'arguments': '{"question":"explain breathing exercises"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 158, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiLu7Qo4AsU6ZM85ZS6bF3kD4Z32v', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--b4b1974b-7f0b-4651-aa78-7693abc4d1f2-0', tool_calls=[{'name': 'rag_tool', 'args

# Prompt

In [ ]:
delegate_agent_prompt = """
You are a Notion Supervisor Agent that strictly follows this workflow:

1. FIRST CHECK: Is this about meditation? (breathing, mindfulness, techniques, relax, stress)
   - YES: Route to meditation_assistant first
   - NO: Proceed to page/block handling

2. FOR MEDITATION CONTENT:
   a) Call meditation_assistant(query=...) and get answer
   b) Create page with notion_page_agent(title=...)
   c) Add content with notion_block_agent(page_id=..., content=meditation_answer)

3. FOR REGULAR CONTENT:
   a) Create page with notion_page_agent(title=...)
   b) Add content with notion_block_agent(page_id=..., content=...)

MEDITATION KEYWORDS TO TRIGGER ASSISTANT:
- meditation, breathe, breathing, mindfulness, relax, calm, stress, focus, technique

STRICT RULES:
- Never proceed to block agent without valid page_id
- Always confirm content was added
- Retry failed steps with simplified content
"""

In [ ]:
# Modified extract_page_id function
def extract_page_id_from_notion_response(response):
    """Improved extraction that handles both API responses and agent messages"""
    if isinstance(response, dict):  # Direct API response
        return response.get("id", "").replace("-", "")

    # For agent messages
    import re
    match = re.search(r'"id":\s*"([a-f0-9-]+)"', str(response)) or \
            re.search(r"page_id\s*=\s*['\"]([a-f0-9]+)['\"]", str(response))
    return match.group(1).replace("-", "") if match else None

In [ ]:
delegate_agent_prompt = """
You are a Notion Supervisor Agent that orchestrates between:
1. Notion Page Agent (creates pages)
2. Notion Block Agent (adds content)
3. Meditation Assistant (provides meditation knowledge)

## WORKFLOW RULES

1. MEDITATION REQUESTS:
   - If request contains: meditation, breathe, breathing, mindfulness, relax, calm, stress, focus, technique
   - FIRST call: `meditation_assistant(query=...)`
   - THEN create page and add content

2. REGULAR PAGE CREATION:
   - Call `notion_page_agent(title=...)`
   - Then `notion_block_agent(page_id=..., content=...)`

3. STRICT ORDER:
   - Never call block agent without valid page_id
   - Verify content exists before adding
   - Retry failed steps with simplified content

## ERROR HANDLING:
- If page creation fails: "Failed to create page - please try again"
- If content missing: "No content to add"
- If any tool fails: "I couldn't complete this request"
"""

supervisor = create_supervisor(
    agents=[notion_page_agent, notion_block_agent, meditation_assistant],
    model=model,
    prompt=delegate_agent_prompt
).compile()

In [ ]:
# Test meditation flow
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(content="Create page 'MindfulBreathing' with meditation techniques")
        ]
    },
    {"recursion_limit": 150}
):
    print(chunk)
    print("---")

{'supervisor': {'messages': [HumanMessage(content="Create page 'MindfulBreathing' with meditation techniques", additional_kwargs={}, response_metadata={}, id='abf8278a-9d79-4552-84b5-1d362c12a7e0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LqY7kZj8Pa5YnIx2AbaRpWEL', 'function': {'arguments': '{"query":"meditation techniques"}', 'name': 'transfer_to_meditation_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 322, 'total_tokens': 343, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiMOLU6m9R14VUc0nPIrEVzcnihxk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--ce67078a-e870-4b

# Prompt 2 Working

In [ ]:
def extract_page_id_from_notion_response(response):
    """Improved extraction that handles both API responses and agent messages"""
    # If response is direct from Notion API
    if isinstance(response, dict):
        return response.get("id", "").replace("-", "")

    # For string responses
    import re
    # Try extracting from URL format first
    url_match = re.search(r"notion\.so/[^-]+-([a-f0-9]{32})", str(response))
    if url_match:
        return url_match.group(1)

    # Then try raw ID format
    id_match = re.search(r"(?:page_)?id\s*[=:]\s*['\"]?([a-f0-9]{32})['\"]?", str(response), re.I)
    return id_match.group(1) if id_match else None

In [ ]:
@tool
def add_notion_paragraph_block(page_id: str, content: str):
    """Properly formatted paragraph block addition"""
    url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
    headers = {
        'Authorization': f'Bearer {NOTION_API_KEY}',
        'Content-Type': 'application/json',
        'Notion-Version': '2022-06-28'
    }

    # Convert content to Notion's block format
    blocks = []
    for paragraph in content.split("\n\n"):
        if paragraph.strip():
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{
                        "type": "text",
                        "text": {"content": paragraph.strip()}
                    }]
                }
            })

    data = {"children": blocks}

    try:
        response = httpx.patch(url, headers=headers, json=data)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

In [ ]:
delegate_agent_prompt = """
You are a Notion Supervisor Agent that strictly follows this workflow:

1. FOR MEDITATION REQUESTS:
   - Call meditation_assistant(query) FIRST
   - Verify response contains actual content
   - Then create page with notion_page_agent(title)
   - Extract page_id from response (must be 32-char hex)
   - Call notion_block_agent(page_id, content)

2. CONTENT VERIFICATION:
   - Before calling block agent:
     * Confirm page_id is valid (32 chars, hex)
     * Confirm content is non-empty plain text
     * Remove any markdown formatting if needed

3. ERROR HANDLING:
   - If page creation fails: "Failed to create page"
   - If content missing: "No content to add"
   - If block addition fails: Retry with simplified text
"""

In [ ]:
# Test with debug output
response = supervisor.invoke({
    "messages": [HumanMessage(
        content="Create 'TestPage' with breathing techniques"
    )]
}, {"debug": True})

# Check the actual Notion page to verify content appears

In [ ]:
# Test with debug output
response = supervisor.invoke({
    "messages": [HumanMessage(
        content="Create 'Mantra' with Gayatri mantra"
    )]
}, {"debug": True})

# Check the actual Notion page to verify content appears

In [ ]:
# Test with debug output
response = supervisor.invoke({
    "messages": [HumanMessage(
        content="Create 'TestPage1' with breathing techniques"
    )]
}, {"debug": True})

# Check the actual Notion page to verify content appears

In [ ]:
# Test with debug output
response = supervisor.invoke({
    "messages": [HumanMessage(
        content="Create 'TestPage2' with meaning of Gayatri Mantra"
    )]
}, {"debug": True})

# Check the actual Notion page to verify content appears

In [ ]:
# First run the operation while capturing the debug output
debug_output = []
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named 'Breathing exercises' with breathing techniques"
            )
        ],
        "config": {"debug": True}
    },
    {"recursion_limit": 150},
):
    debug_output.append(chunk)
    print(chunk)
    print("\n")

# Then print the full debug information
print("\n=== DEBUG OUTPUT ===")
for i, chunk in enumerate(debug_output):
    print(f"\n--- Step {i+1} ---")
    print(chunk)

{'supervisor': {'messages': [HumanMessage(content="Create a new page named 'Breathing exercises' with breathing techniques", additional_kwargs={}, response_metadata={}, id='f81210d9-6904-4d75-bd99-1c7c5e26fb89'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XVBU3A8cMKmd72STnNnZRvs3', 'function': {'arguments': '{"query":"breathing techniques"}', 'name': 'transfer_to_meditation_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 272, 'total_tokens': 293, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiMg2Dbo2OqsXpZz4C0BOzcnvKUqW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--931